## Scraping multiple pages in AirBNB

In [1]:
import time
import requests
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
url = "https://www.airbnb.com.sg/s/Malaysia/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&place_id=ChIJrzBnX5fTNDAR2BwhKJOWRQc&date_picker_type=calendar&checkin=2022-10-14&checkout=2022-10-16&adults=2&source=structured_search_input_header&search_type=filter_change"

In [3]:
page = requests.get(url)
page

<Response [200]>

In [4]:
soup = BeautifulSoup(page.text, 'lxml')

In [5]:
#Creates a dataframe using the column headers from our table
df = pd.DataFrame({'Links': [''],'Title': [''], 'Details': [''],'Price': [''], 'Rating': [''], 'Bed': ['']})
df

,Links,Title,Details,Price,Rating,Bed
0,,,,,,


In [ ]:
# Modified cell

%%time

while True:
    postings = soup.find_all('div', class_ = 'c4mnd7m dir dir-ltr')
    for post in postings:
        try:
            link = post.find('a', class_ = 'ln2bl2p dir dir-ltr').get('href')
            link_full = 'https://www.airbnb.com.sg' + link
            title =  post.find('div', class_ = 't1jojoys dir dir-ltr').text
            price = post.find('span', class_ = '_tyxjp1').text.strip().replace("\xa0SGD", "")
            rating = post.find('span', class_ = 'ru0q88m dir dir-ltr').text
            details = post.find('span', class_ = 'tjbvqj3 dir dir-ltr').text
            bed = post.find_all('div', class_ = 'f15liw5s s1cjsi4j dir dir-ltr')[1].text
            df = pd.concat([df, pd.DataFrame.from_records([{'Links': link_full,'Title': title, 'Details': details, 'Price': price, 'Rating': rating, 'Bed': bed }])], ignore_index=True)
        except:
            pass
    

    try:
        next_page = soup.find('a', {'aria-label':'Next'}).get('href')
    except AttributeError:
        break
        
    next_page_full = 'https://www.airbnb.com.sg/' + next_page
    next_page_full
    
    url = next_page_full
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    time.sleep(120)


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 1 to 100
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Links    100 non-null    object
 1   Title    100 non-null    object
 2   Details  100 non-null    object
 3   Price    100 non-null    object
 4   Rating   100 non-null    object
 5   Bed      100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB


In [7]:
df.head()

,Links,Title,Details,Price,Rating,Bed
0,,,,,,
1,https://www.airbnb.com.sg/rooms/53818538?adult...,Holiday home in Melaka,✦ATTIC✦Sky Garden Private Pool Luxury Family S...,$330,4.96 (47),3 beds
2,https://www.airbnb.com.sg/rooms/5302083?adults...,Villa in Langkawi,Alamanda Hilltop Villa - Stunning Sunset View,$154,4.77 (239),1 queen bed
3,https://www.airbnb.com.sg/rooms/39845779?adult...,Apartment in Melaka,✦ATTIC✦ Peak of MLK Seaview Modern Studio [TV ...,$293,4.84 (136),1 king bed
4,https://www.airbnb.com.sg/rooms/39844734?adult...,Apartment in Melaka,✦ATTIC✦Peak of MLK Seaview [Massage Chair&TVBOX],$293,4.87 (137),1 king bed


In [8]:
# Remove 1st row
df = df.iloc[1: , :]
df.head()

,Links,Title,Details,Price,Rating,Bed
1,https://www.airbnb.com.sg/rooms/53818538?adult...,Holiday home in Melaka,✦ATTIC✦Sky Garden Private Pool Luxury Family S...,$330,4.96 (47),3 beds
2,https://www.airbnb.com.sg/rooms/5302083?adults...,Villa in Langkawi,Alamanda Hilltop Villa - Stunning Sunset View,$154,4.77 (239),1 queen bed
3,https://www.airbnb.com.sg/rooms/39845779?adult...,Apartment in Melaka,✦ATTIC✦ Peak of MLK Seaview Modern Studio [TV ...,$293,4.84 (136),1 king bed
4,https://www.airbnb.com.sg/rooms/39844734?adult...,Apartment in Melaka,✦ATTIC✦Peak of MLK Seaview [Massage Chair&TVBOX],$293,4.87 (137),1 king bed
5,https://www.airbnb.com.sg/rooms/34880764?adult...,Villa in Langkawi,Villa 4 - Private Pool WOW Holiday Homes,$530,4.82 (67),4 beds


In [9]:
#exports the data as a csv
df.to_csv('AiBnb_data.csv', index=False)
